# 诗歌生成

# 数据处理

In [1]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets

start_token = 'bos'
end_token = 'eos'

def process_dataset(fileName):
    examples = []
    with open(fileName, 'r', encoding='utf-8') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
            
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('./poems.txt')
    ds = tf.data.Dataset.from_generator(lambda: [ins for ins in instances], 
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

# 模型代码， 完成建模代码

In [2]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(self.v_sz, 64, 
                                                    mask_zero=True)
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
        
    @tf.function
    def call(self, inp_ids):
        '''
        此处完成建模过程，可以参考Learn2Carry
        '''
        x = self.embed_layer(inp_ids)
        x = self.rnn_layer(x)
        if isinstance(x, tuple):
            x = x[0]
        logits = self.dense(x)
        return logits
    
    @tf.function
    def get_next_token(self, x, state):
        '''
        shape(x) = [b_sz,] 
        '''
    
        inp_emb = self.embed_layer(x) #shape(b_sz, emb_sz)
        h, state = self.rnncell.call(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state

## 一个计算sequence loss的辅助函数，只需了解用途。

In [3]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [ ]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    '''
    完成一步优化过程，可以参考之前做过的模型
    '''
    with tf.GradientTape() as tape:
        logits = model.call(x)
        loss = compute_loss(logits, y, seqlen)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss

# 训练优化过程

In [5]:
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)

for epoch in range(10):
    loss = train(epoch, model, optimizer, train_ds)

epoch 0 : loss 8.821366
epoch 1 : loss 6.6369386
epoch 2 : loss 6.0856543
epoch 3 : loss 5.8118787
epoch 4 : loss 5.619367
epoch 5 : loss 5.5959487
epoch 6 : loss 5.4694753
epoch 7 : loss 5.3246717
epoch 8 : loss 5.2207975
epoch 9 : loss 5.1960692


# 生成过程

In [60]:
def gen_sentence(start_words="一"):
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]

    start_tokens = [word2id['bos']]
    for char in start_words:
        if char in word2id:
            start_tokens.append(word2id[char])
        else:
            start_tokens.append(word2id['UNK'])
    
    for token in start_tokens[1:]:
        cur_token = tf.constant([token], dtype=tf.int32)
        _, state = model.get_next_token(cur_token, state)
    
    collect = start_tokens.copy()
    for _ in range(50 - len(start_words)):
        cur_token = tf.constant([collect[-1]], dtype=tf.int32)
        next_token, state = model.get_next_token(cur_token, state)
        collect.append(next_token.numpy()[0])
    
    poem = [id2word[t] for t in collect if id2word[t] not in ['bos', 'eos']]
    return ''.join(poem)
print(gen_sentence(start_words="月"))

月。然不可怜，不得无人间。有不可见，不知何处归。然不可见，不得一年年。有无人事，何人不可怜。


# 报告

[PDF](./RNN.pdf)

## 循环神经网络(RNN)

### RNN、LSTM、GRU 模型解释

1. **RNN（循环神经网络）**
   - **原理**：通过隐藏状态传递序列信息，每个时间步接收当前输入和前一隐藏状态，输出当前隐藏状态。
   - **缺点**：梯度消失/爆炸问题严重，难以捕捉长期依赖。
2. **LSTM（长短期记忆网络）**
   - **改进**：引入输入门、遗忘门、输出门，控制信息流动。
   - **优点**：通过门控机制缓解梯度消失，能有效学习长期依赖。
3. **GRU（门控循环单元）**
   - **改进**：合并 LSTM 的输入门和遗忘门为“更新门”，简化结构。
   - **优点**：参数更少，训练更快，性能接近 LSTM。

------

### 诗歌生成过程

1. **数据处理**
   - 从文本文件加载诗歌，添加 `bos`（开始）和 `eos`（结束）标记。
   - 构建词汇表，将字符映射为索引（ID）。
   - 将诗歌转换为索引序列，并进行填充（Padding）和分批（Batching）。
2. **模型构建**
   - **嵌入层**：将字符索引转换为稠密向量。
   - **RNN层**（如 `SimpleRNN/LSTM/GRU` ）：处理序列数据，捕捉上下文信息。
   - **全连接层**：输出每个时间步的字符概率分布。
3. **训练**
   - 使用交叉熵损失函数，通过反向传播优化模型参数。
   - 每个批次输入为 `x[:, :-1]`，标签为 `x[:, 1:]`，实现自回归训练。
4. **生成**
   - 从起始词（如“日”）开始，逐步预测下一个字符。
   - 使用贪心搜索（取概率最高的字符）或采样策略生成文本，直到遇到 `eos` 或达到最大长度。

------

### 生成诗歌截图

- **一**：![一](./img/一.png)
- **日**：![日](./img/日.png)
- **山**：![山](./img/山.png)
- **红**：![红](./img/红.png)
- **夜**：![夜](./img/夜.png)
- **湖**：![湖](./img/湖.png)
- **海**：![海](./img/海.png)
- **月**：![月](./img/月.png)

------

### 实验总结

1. **训练效果**
   - 损失从初始值 8.821 下降至 5.196，模型逐渐收敛。
   - 生成诗歌的连贯性随训练轮次增加而提升。
2. **问题分析**
   - 简单 RNN 存在梯度消失，生成诗歌较短且重复。
   - 可替换为 LSTM 或 GRU 以提升长文本生成能力。
3. **改进方向**
   - 使用更大的数据集和更深层的网络。
   - 引入注意力机制或预训练模型。
4. **结论**
   RNN 在诗歌生成任务中能学习基本模式，但更复杂的模型（如 `LSTM` ）和策略（如 `Beam Search` ）可显著提升生成质量。